In [1]:
// Add the maven dependencies
%maven ai.djl:api:0.28.0
%maven ai.djl:model-zoo:0.28.0
%maven ai.djl.mxnet:mxnet-engine:0.28.0
%maven ai.djl.mxnet:mxnet-model-zoo:0.28.0
%maven org.slf4j:slf4j-simple:1.7.36

In [2]:
import java.awt.image.*;
import java.nio.file.*;
import java.util.*;
import java.util.stream.*;
import ai.djl.*;
import ai.djl.basicmodelzoo.basic.*;
import ai.djl.ndarray.*;
import ai.djl.modality.*;
import ai.djl.modality.cv.*;
import ai.djl.modality.cv.util.NDImageUtils;
import ai.djl.translate.*;

In [3]:
// Check out a sample Mnist example
var img = ImageFactory.getInstance().fromUrl("https://resources.djl.ai/images/0.png");
img.getWrappedImage();

In [15]:
// Load the model we trained in train_img_classification
Path modelDir = Paths.get("build/mlp");
Model model = Model.newInstance("mlp");
model.setBlock(new Mlp(28 * 28, 10, new int[] {128, 64}));
model.load(modelDir);

In [30]:
Translator<Image, Classifications> translator = new Translator<Image, Classifications>() {
    
    @Override
    public NDList processInput(TranslatorContext ctx, Image input) {
        // Convert Image to NDArray
        NDArray array = input.toNDArray(ctx.getNDManager(), Image.Flag.GRAYSCALE);
        return new NDList(NDImageUtils.toTensor(array));
    }

    @Override
    public Classifications processOutput(TranslatorContext ctx, NDList list) {
        // Create a Classifications with the output probabilities
        NDArray probabilities = list.singletonOrThrow().softmax(0);
        List<String> classNames = IntStream.range(0, 10).mapToObj(String::valueOf).collect(Collectors.toList());
        return new Classifications(classNames, probabilities);
    }

    @Override
    public Batchifier getBatchifier() {
        // The Batchifier describes how to combine a batch together
        // Stacking, the most common batchifier, takes N [X1, X2, ...] arrays to a single [N, X1, X2, ...] array
        return Batchifier.STACK;
    }
};

In [31]:
// Using the translator, we will create a new Predictor. 
// The predictor is the main class to orchestrate the inference process. 
// During inference, a trained model is used to predict values, often for production use cases. 
// The predictor is NOT thread-safe, so if you want to do prediction in parallel, 
// you should call newPredictor multiple times to create a predictor object for each thread.
var predictor = model.newPredictor(translator);
var classifications = predictor.predict(img);

classifications

[
	{"class": "0", "probability": 0.99983}
	{"class": "7", "probability": 0.00008}
	{"class": "2", "probability": 0.00004}
	{"class": "6", "probability": 0.00002}
	{"class": "5", "probability": 4.7e-06}
]